# **Introduction to LangChain**

**Build Applications That Can Reason**

LangChain is a framework for developing applications powered by large language models (LLMs).

### **Architecture**
LangChain is a framework that consists of a number of packages.

<img src="images/langchain_stack_updated.png">

**langchain-core**  
This package contains base abstractions for different components and ways to compose them together. The interfaces for core components like **chat models**, **vector stores**, **tools** and more are defined here. No third-party integrations are defined here. The dependencies are very lightweight.

**langchain**  
The main langchain package contains chains and retrieval strategies that make up an application's cognitive architecture. These are NOT third-party integrations. All chains, agents, and retrieval strategies here are NOT specific to any one integration, but rather generic across all integrations.

**Integration packages**  
Popular integrations have their own packages (e.g. langchain-openai, langchain-anthropic, etc) so that they can be properly versioned and appropriately lightweight.

**langchain-community**  
This package contains third-party integrations that are maintained by the LangChain community. Key integration packages are separated out (see above). This contains integrations for various components (chat models, vector stores, tools, etc). All dependencies in this package are optional to keep the package as lightweight as possible.

**langgraph**  
langgraph is an extension of langchain aimed at building robust and stateful multi-actor applications with LLMs by modeling steps as edges and nodes in a graph.

LangGraph exposes high level interfaces for creating common types of agents, as well as a low-level API for composing custom flows.

## **Chains - Prompt Template + Model + Output Parser**

**Prompt Template**  
- Prompt Templates are used to convert raw user input to a better input to the LLM. Templates allow us to easily configure and modify our input prompts to LLM calls.

**Model**  
- LLMs handle various language operations such as translation, summarization, question answering, and content creation.

**Output Parser**  
- It's often much more convenient to work with strings. We can add a simple output parser to convert the chat message to a string.

**Chains**  
- The `|` symbol chains together the different components feeds the output from one component as input into the next component.


<img src="images/langchain_LCEL.JPG">

In [1]:
# # Install langchain-openai

# ! pip install langchain-openai

In [2]:
# Setup API Key

f = open('keys/.openai_api_key.txt')

OPENAI_API_KEY = f.read()

### **Models**

In [3]:
# Import OpenAI ChatModel
from langchain_openai import ChatOpenAI

# Set the OpenAI Key and initialize a ChatModel
chat_model = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

# Creating a Prompt
prompt = "What is Feature Engineering?"

# Printing the output of model
print(chat_model.invoke(prompt))

/Users/kanavbansal/Developer/.env_jupyter/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


content='Feature engineering is the process of using domain knowledge to select, modify, or create new features (variables or attributes) from raw data to improve the performance of machine learning models. It is a crucial step in the machine learning workflow, as the quality and relevance of features can significantly influence the accuracy and effectiveness of a model.\n\nHere are some key aspects of feature engineering:\n\n1. **Feature Selection**: Identifying and selecting the most relevant features from the dataset. This can involve removing redundant or irrelevant features that do not contribute useful information for the model.\n\n2. **Feature Creation**: Generating new features from existing ones to provide more information to the model. This can include mathematical transformations, aggregating data points, or creating interaction terms between features.\n\n3. **Encoding**: Converting categorical variables into numerical formats that can be understood by machine learning algor

### **Prompts**

Input to a model can be a **string** or **chat messages**.

Prompt Template - Prompt templates are used to convert raw user input to a better input to the LLM or ChatModels.

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI Tutor with expertise in Data Science and Artificial Intelligence. "),
    ("human", "What is {topic}?"),
])

In [5]:
prompt_template.input_variables

['topic']

### **Chains**

**The `|` symbol chains together the different components feeds the output from one component as input into the next component.**

In [7]:
chain = prompt_template | chat_model

raw_input = {'topic': "NLP"}

chain.invoke(raw_input)

AIMessage(content='Natural Language Processing (NLP) is a subfield of artificial intelligence (AI) and computational linguistics that focuses on the interaction between computers and humans through natural language. The goal of NLP is to enable computers to understand, interpret, generate, and respond to human language in a way that is both meaningful and useful.\n\nNLP encompasses a variety of tasks and applications, including but not limited to:\n\n1. **Text Analysis**: Analyzing and extracting information from text data, such as sentiment analysis, topic modeling, and named entity recognition.\n\n2. **Machine Translation**: Automatically translating text from one language to another (e.g., Google Translate).\n\n3. **Speech Recognition**: Converting spoken language into text (e.g., virtual assistants like Siri or Alexa).\n\n4. **Text Generation**: Producing coherent and contextually relevant text based on a prompt (e.g., chatbots, content generation).\n\n5. **Question Answering**: De

### **Output Parser**

**The output of a ChatModel (and therefore, of this chain) is a AI Message. However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string.**

In [8]:
# Output Parsing

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [9]:
# Modifying the chain: Adding output_parser
chain = prompt_template | chat_model | output_parser

raw_input = {"topic": "deep learning"}

chain.invoke(raw_input)

'Deep learning is a subset of machine learning, which itself is a branch of artificial intelligence (AI). It involves the use of neural networks with many layers (hence the term "deep") to model and understand complex patterns in data. Here are some key points to understand about deep learning:\n\n1. **Neural Networks**: At its core, deep learning utilizes artificial neural networks, which are computational models inspired by the way biological neural networks in the human brain work. A typical neural network consists of an input layer, multiple hidden layers, and an output layer.\n\n2. **Layers**: The "depth" in deep learning comes from having multiple hidden layers between the input and output layers. Each layer consists of nodes (or neurons) that apply transformations to the input data and pass the resulting output to the next layer.\n\n3. **Feature Learning**: One of the strengths of deep learning is its ability to automatically learn features from raw data. This contrasts with tra

## **Example: Create an AI Tutor App that uses Prompts and Chat internally to give Python Implementation tutorial for Data Science topics**

In [10]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [11]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Constructing System Prompt
system_prompt = SystemMessagePromptTemplate.from_template("""
You are a friendly AI Tutor with expertise in Data Science and AI who tells step by step Python Implementation for topics asked by user.
""")

# Constructing Human Prompt
human_prompt = HumanMessagePromptTemplate.from_template("Tell me a python implementation for {topic_name}.")

# Compiling Chat Prompt
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

chat_prompt

ChatPromptTemplate(input_variables=['topic_name'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nYou are a friendly AI Tutor with expertise in Data Science and AI who tells step by step Python Implementation for topics asked by user.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic_name'], input_types={}, partial_variables={}, template='Tell me a python implementation for {topic_name}.'), additional_kwargs={})])

In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [13]:
chain = chat_prompt | chat_model | output_parser

In [14]:
raw_input = {"topic_name": "Logistic Regression"}

output = chain.invoke(raw_input)

print(output)

Sure! Here is a step-by-step Python implementation for Logistic Regression using the popular library `scikit-learn`:

1. Import the necessary libraries:
```python
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.metrics import accuracy_score
```

2. Load a sample dataset (for demonstration purposes, we will use the Iris dataset):
```python
iris = datasets.load_iris()
X = iris.data
y = iris.target
```

3. Split the dataset into training and testing sets:
```python
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

4. Initialize the Logistic Regression model and fit it to the training data:
```python
model = LogisticRegression()
model.fit(X_train, y_train)
```

5. Make predictions on the test set:
```python
predictions = model.predict(X_test)
```

6. Evaluate the model performance by calculating the accuracy:
```python
accuracy =